In [ ]:
import numpy as np
from numpy import fft

import scipy
from scipy.optimize import newton, curve_fit
from scipy.ndimage import center_of_mass
from skimage import filters, measure

from numba import njit
import glob
import os

import matplotlib.pyplot as plt
import h5py
import yt

figures = "figures/"
figheight = 4
os.makedirs(figures, exist_ok = True)
DPI = 300

import imageio.v3 as iio
import skimage as ski

This notebook is used to characterize the data obtained to validate the implementation of thermodynamically consistent fluctuations within a multicomponent lattice boltzmann method based on a free energy formulation. The implementation is written in C++ based on AMReX. The free energy formulation is built using the free energy model proposed in [Swift et al. 1996](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.54.5041) which utilizes a square gradient free energy functional and a bulk free energy defined as two ideal fluids interacting with each other. The modified equilibrium distribution proposed in Swift et al. is utilized.

We follow the procedure to develop a formalism fo the fluctuations using the process detailed in [Gross et al. 2010](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.82.056714). They propose spatial correlations of the noise in k space, resulting in a better match to theory especially at higher k values. They state an ansatz which specifies the form of the structure factor of the noise, allowing for the calculation of a covariance matrix that is dependent upon the thermodynamic model utilized. 

To validate the implementation, the equilibration ratios of the density, order parameter and velocity are calculated for a mixed system of equal volume fractions of each fluid. Next, the interfacial fluctuations are calculated using Eqaution 3.11 in calculated by [Grant and Desai 1983](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.27.2577). Finally, the surface tension calculated from the fluctuations of the shape of a droplet will be compared to that calculated using the Young-Laplace equation as defined in [Benayad et al. 2020](https://doi.org/10.1021/acs.jctc.0c01064).

# Helper functions

## Data I/O

### `extract_data`

AMReX can output datafiles with type `h5`. There are 38 fields of data representing the 19 moments of each distribution equation used to describe the hydrodynamics and non-ideal mixing. The ordering of these moments are, $\rho, \phi, v_x, v_y, v_z, \phi v_x, \phi v_y, \phi v_z, mf4 ... mf18, mg4 ... mg18$

**Input**
1. `h5_filepath`: `str` where the `.h5` file as output from AMReX is located
2. `dims`: Dimensions of the simulation box input as a `list` or `np.array`
3. `begin`: `int` defining the index upon which to start reading the moment fields. 
4. `end`: `int` defining the index upon which to end reading the moment fields (non inclusive).
5. `nVars`: `int` defining the number of data fields that the output file contains

In [ ]:
def extract_data(h5_filepath, dims, begin = 0, end = 1, nVars = 38):
    if type(dims) == list:
        dims = np.array(dims)

    with h5py.File(h5_filepath,'r') as h5f:
        keys = list(h5f.keys())
        test = h5f[keys[-1]]
        keys = list(test.keys())
        
        # boxDims = test[keys[1]][()]
        # # print(boxDims)
        domain_decomp = test[keys[1]][()]
        data = test[keys[2]][()]
    
    output = np.zeros((end - begin, *dims))
    decomp_box_size = [int(domain_decomp[0][5]-domain_decomp[0][2]+1),
                       int(domain_decomp[0][4]-domain_decomp[0][1]+1),
                       int(domain_decomp[0][3]-domain_decomp[0][0]+1)]
    sz = 1
    for i in decomp_box_size: sz *= i
    revdims = np.flip(dims)

    for i in range(begin, end):
        currVar = np.zeros(revdims)
        for j in range(0, len(domain_decomp)):
            curr_decomp = domain_decomp[j]
            slc = np.s_[curr_decomp[2]:curr_decomp[5]+1, curr_decomp[1]:curr_decomp[4]+1, curr_decomp[0]:curr_decomp[3]+1]
            currVar[slc] = data[(i + j*nVars)*sz:(i+1 + j*nVars)*sz].reshape(*decomp_box_size)
    
        output[i - begin] = np.moveaxis(currVar, [0, -1], [-1, 0])

    return output

## LBM and thermodynamic model

### `lattice_fourier_laplacian`

This function defines the laplacian operator as it appears in k space, defined as the fourier transform of the expression $\sum_{i \neq 0} [\rho(r + c_i) + \rho(r - c_i) - 2\rho(r)]/c_s^2$ where $c_s^2$ represents the speed of sound of the lattice kernel used. In practice, this expression becomes, $\frac{\frac{2}{9}[\cos{kx} + \cos{ky} + \cos{kz}] + \frac{2}{9}[\cos{kx}\cos{ky} + \cos{ky}\cos{kz} + \cos{kx}\cos{kz}] - \frac{4}{3}}{c_s^2}$. To be used as a substitute for any term that contains $\nabla^2$ in real space.

**Input**
1. `kx`: Wave vector in the x direction. Can be passed as a `float` or `np.array`
1. `ky`: Wave vector in the y direction. Can be passed as a `float` or `np.array`
1. `kz`: Wave vector in the z direction. Can be passed as a `float` or `np.array`

**Output**
1. `k2`: Lattice laplacian value in k space. Output as `float` or `np.array` depending on input

In [ ]:
def lattice_fourier_laplacian(kx, ky, kz):
    expr1 = np.cos(kx) + np.cos(ky) + np.cos(kz)
    expr2 = np.cos(kx)*np.cos(ky) + np.cos(ky)*np.cos(kz) + np.cos(kx)*np.cos(kz)
    out = 2/9*expr1 + 2/9*expr2 - 4/3
    cs2 = 1/3
    k2 = -out/cs2
    return k2

### `swift_et_al_1996_thermodynamic_model`

Defines the thermodynamic parameters of the swift et al. thermodynamic model as defined in [Swift et al. 1996](https://journals.aps.org/pre/pdf/10.1103/PhysRevE.54.5041).

**Input**
1. `density`: `float` representing the density of the system
2. `C0`: `float` representing the order parameter value
3. `chi`: `float` a value corresponding to the width of the double well
4. `T`: `float` representing the depth of the double well
5. `kappa`: `float` representing the strength of non ideal mixing

**Output**
1. `model class` with various operations

In [ ]:
class swift_et_al_1996_thermodynamic_model:
    def __init__(self, density = 1, C0 = 0, chi = 0.4, T = 0.25, kappa = 0.01):
        self.chi = chi
        self.T = T
        self.kappa = kappa
        self.rho = density
        self.C0 = C0

    def sound_speed_square(self):
        out = self.T
        return out

    def cs2k(self, kx = 0, ky = 0, kz = 0):
        thermal_cs2 = self.sound_speed_square()
        k2 = lattice_fourier_laplacian(kx, ky, kz)
        out = thermal_cs2 + k2*self.kappa
        return out
    
    def calculate_df_dphi(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = -chi/2.*(phi/rho) + T/2.*np.log((1. + phi/rho)/(1. - phi/rho))
        return out

    def calculate_dmup_drho(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = -T*phi/(rho**2 - phi**2) + chi/2*phi/(rho**2)
        return out

    def calculate_dmup_dphi(self):
        rho = self.rho
        phi = self.C0
        chi = self.chi
        T   = self.T
        out = T*rho/(rho**2 - phi**2) - chi/(2*rho)
        return out
    
    def mu_ck(self, kx = 0, ky = 0, kz = 0):
        ref_state = self.calculate_dmup_dphi()
        k2 = lattice_fourier_laplacian(kx, ky, kz)
        out = ref_state + k2*self.kappa
        return out

### `interface_height`

To calculate the interface height, three methods are defined in separate functions. These detail three ways to calculate the interface height fluctuations. The first, termed as `direct_old` is a direct implementation identifying the interface height based on the gradient of the order parameter profile. The second, termed `direct` fits the point before and after the location of the interface to a linear expression, followed by identifying the root of the linear expression. The final variant termed `profile-fit`, fits the profile of the order parameter to a profile, $\phi = \phi_0\tanh{\frac{x - b}{\sqrt{2} \xi}}$ profile and uses the fit parameter for $b$. `interface_height` wraps these 3 implementations into a single function so as to make using each implementation easier.

**Input**

1. `density`: `1D np.array` of the order parameter at each slice of the 3D array
2. `chi`: `float` a value corresponding to the width of the double well
3. `T`: `float` representing the depth of the double well
4. `kappa`: `float` representing the strength of non ideal mixing
5. `method`: `str` of which method to use to calculate the interface height. Throws a valueError if invalid method is input

In [ ]:
def fb(phi, l, T, rho = 1):
    model = swift_et_al_1996_thermodynamic_model(rho, phi, l, T, kappa = 0)
    out = model.calculate_df_dphi()
    return out

def ih_direct_old(profile):
    grad_field_dx = np.abs(np.gradient(profile, axis = 0))
    idx_max = np.argmax(grad_field_dx, axis = 0)
    return idx_max

def ih_direct(profile):
    nx, ny, nz = profile.shape
    out = np.zeros((ny, nz))
    intLoc = (nx - 1)/2

    xLo = int(np.floor(intLoc))
    xHi = int(np.ceil(intLoc))
    
    for y in range(ny):
        for z in range(nz):
            slc = profile[:, y, z]
            xslc = [xLo, xHi]
            yslc = [slc[xLo], slc[xHi]]
            out[y, z] = np.roots(np.polyfit(xslc, yslc, 1))[0]
    return out

def ih_profile_fit(profile, chi, T, kappa):
    nx, ny, nz = profile.shape
    out = np.zeros((ny, nz))
    phi0 = newton(fb, x0 = (0.6), args = (chi, T))
    xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
    fit_func = lambda x, b:phi0*np.tanh((x - b)/(np.sqrt(2)*xi))

    xraw = np.arange(0, nx, 1)

    for y in range(ny):
        for z in range(nz):
            slc = profile[:, y, z]
            popt, pcov = curve_fit(fit_func, xraw, slc, p0 = [nx//2])
            out[y, z] = popt[0]
    return out

def interface_height(density, chi, T, kappa, method = "direct", zero = False):
    height_func = np.zeros(density.shape[1:])

    nx, ny, nz = density.shape
    lo_min = nx//4
    hi_max = 3*(nx//4)
    zero_factor = (nx/2 - 1)/2
    yraw = density[lo_min:hi_max, :, :]

    if method == 'direct':
        height_func = ih_direct(yraw)# - zero_factor
    elif method == "profile_fit":
        height_func = ih_profile_fit(yraw, chi, T, kappa)# - zero_factor
    elif method == "direct_old":
        height_func = ih_direct_old(yraw)# - zero_factor
    else:
        raise ValueError(f'{method} is invalid to calculate interface height')
    
    height_func = height_func - zero_factor if zero else height_func
    
    return height_func

## Plotting

### `spherically_averaged_structure_factor`

**Input**
1. `data`: `np.array` containing the structure factor data for for a moment
2. `thermo_model`: `thermodynamics class` for calculation of various parameters directly relevant to the thermodynamic model to be utilized
3. `scale_factor`: `float` or `np.array` that the data is divided by
4. `func`: `lambda function` that defines some further operations to scale data. Used for spatial correlations
5. `shift`: `bool` whether to shift the zero point of the frequencies calculated by `np.fft.fftfreq` to the same order as numpy arrays or as the output of fftw
6. `cs`: `bool` whether to use the speed of sound or chemical potential

In [ ]:
def spherically_averaged_structure_factor(data, thermo_model, scale_factor = 1, func = None, shift = True, cs = True):
    L = min(data.shape)
    S = data.copy()

    if shift:
        freqs = fft.fftshift(fft.fftfreq(L))
    else:
        freqs = fft.fftfreq(L)
    if len(data.shape) == 3:
        kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
        k = np.stack([kx, ky, kz], axis = -1)
    elif len(data.shape) == 2:
        kx, ky = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L]]), indexing='ij')
        k = np.stack([kx, ky], axis = -1)
     
    k1 = np.linalg.norm(k, axis=-1).flatten()
    
    if func is not None:
        if cs:
            S = func(S, thermo_model.cs2k(kx, ky, kz))
        else:
            S = func(S, thermo_model.mu_ck(kx, ky, kz))
    S /= scale_factor

    # test[slc] /= test.sum()
    # S[L//2, L//2, L//2] /= S.sum()

    S1 = S.flatten()
    kmin = 2*np.pi/L # sampling frequency
    where = np.s_[:]#np.where(k1<=kmax)
    bins = np.arange(L//2+1)*kmin # kmax+1 for bin_edges: len(bins)=len(hist)+1
    
    shells = np.histogram(k1[where], bins, weights=S1[where])[0]
    counts = np.histogram(k1[where], bins)[0]
    return (bins[:-1]+bins[1:])/2, shells/counts

### `radial_equilibration`

**Input**
1. `data`: `np.array` containing the structure factor data for for a moment
2. `thermo_model`: `thermodynamics class` for calculation of various parameters directly relevant to the thermodynamic model to be utilized
3. `scale_factor`: `float` or `np.array` that the data is divided by
4. `func`: `lambda function` that defines some further operations to scale data. Used for spatial correlations
5. `shift`: `bool` whether to shift the zero point of the frequencies calculated by `np.fft.fftfreq` to the same order as numpy arrays or as the output of fftw
6. `cs`: `bool` whether to use the speed of sound or chemical potential

In [ ]:
def cart2sph(x,y,z):
    azimuth = np.arctan2(y,x)
    elevation = np.arctan2(z,np.sqrt(x**2 + y**2))
    r = np.sqrt(x**2 + y**2 + z**2)
    return r, azimuth, elevation

def sph2cart(azimuth,elevation,r):
    x = r * np.cos(elevation) * np.cos(azimuth)
    y = r * np.cos(elevation) * np.sin(azimuth)
    z = r * np.sin(elevation)
    return x, y, z

def radial_equilibration(data, thermo_model, radius = 1, scale_factor = 1, func = None, cs = True):
    S = data.copy()
    L = min(S.shape)
    freqs = fft.fftshift(fft.fftfreq(L))
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

    r, t, p = cart2sph(kx, ky, kz)

    if func is not None:
        if cs:
            S = func(S, thermo_model.cs2k(kx, ky, kz))
        else:
            S = func(S, thermo_model.mu_ck(kx, ky, kz))
    S /= scale_factor
    # S[L//2, L//2, L//2] /= S.sum()
    
    idxs = np.isclose(r, radius, atol = 2*np.pi/L)
    t = t[idxs]
    p = p[idxs]
    out = S[idxs]

    return t, p, out

### `make_bins`

**Input**
1. `to_bin1`: `np.array` containing data of a list to be binned
2. `binsize`: `int` defining how many size of the output array. `len(to_bin1)/binsize` is the number of points averaged over to generate the output
3. `to_bin2`: `np.array` containing a 2nd list to be be binned (optional)

In [ ]:
def make_bins(to_bin1, binsize, to_bin2 = None):
    bins = np.linspace(to_bin1.min(), to_bin1.max(), binsize)

    out1 = np.zeros(binsize)
    shell = np.digitize(to_bin1, bins = bins, right = True)
    np.add.at(out1, shell, to_bin1)
    unique, counts = np.unique(shell, return_counts=True)
    out1 = out1[unique]
    out1 /= counts

    if to_bin2 is None:
        return bins, out1
    else:
        out2 = np.zeros(binsize)
        np.add.at(out2, shell, to_bin2)
        unique, counts = np.unique(shell, return_counts=True)
        out2 = out2[unique]
        out2 /= counts
        return out1, out2

## Droplets

### `pressure_tensor`

**Input**
1. `profile`: `np.array` of shape (nx, ny, nz, 2) containing density and order parameter data in the 0th and 1st index of the last dimension respectively
2. `T`: `float` defining a thermodynamic parameter
3. `kappa`: `float` defining a thermodynamic parameter related to surface tension

In [ ]:
def pressure_tensor(profile, T, kappa):
    rho = profile[0]
    phi = profile[1]
    dims = len(rho.shape)

    out = np.zeros((*rho.shape, dims, dims))

    for i in range(dims):
        for j in range(dims):
            out[..., i, j] += kappa*np.gradient(rho, axis = i)*np.gradient(rho, axis = j) + kappa*np.gradient(phi, axis = i)*np.gradient(phi, axis = j)
            if i == j:

                laplacian = rho*np.sum([np.gradient(rho, 2, axis = ax) for ax in range(dims)], axis = 0) + phi*np.sum([np.gradient(phi, 2, axis = ax) for ax in range(dims)], axis = 0)
                gradients = np.sum([np.gradient(rho, 1, axis = ax)**2 for ax in range(dims)], axis = 0) + np.sum([np.gradient(phi, 1, axis = ax)**2 for ax in range(dims)], axis = 0)
                out[..., i, j] += rho*T - kappa*laplacian - kappa/2*gradients

    return out

### `pressure_jump`

**Input**
1. `pressure`: `np.ndarray` of shape `[nx, ny, nz]`

In [ ]:
def pressure_jump(pressure):
    nx, ny, nz = pressure.shape
    center_slc = np.s_[nx//2-1:nx//2+2, ny//2-1:ny//2+2, nz//2-1:nz//2+2]
    edge_slc = np.s_[0:nx:nx-1, 0:ny:ny-1, 0:nz:nz-1]
    dP = pressure[center_slc].mean() - pressure[edge_slc].mean()
    return dP

### `pressure_jump`

**Input**
1. `pressure`: `np.ndarray` of shape `[nx, ny, nz]`

In [ ]:
def droplet_mass(OutArray):
    sum = np.sum(OutArray)
    return sum

### `droplet_radius_mass`

**Input**
1. `density`: `np.ndarray` of shape `[nx, ny, nz]` which holds order parameter data of the droplet
2. `Vp`: `float` of particle volume if particles are used
3. `np_sphere`: `float` of number of particles if particles are used
4. `rho_sphere`: `float` of particle density if particles are used

In [ ]:
def droplet_radius_mass(density, Vp = 0, np_sphere = 0, rho_sphere = 1):
    nx, ny, nz = density.shape
    center_slc = np.s_[nx//2-1:nx//2+2, ny//2-1:ny//2+2, nz//2-1:nz//2+2]
    edge_slc = np.s_[0:nx:nx-1, 0:ny:ny-1, 0:nz:nz-1]
    if isinstance(density, int):
        return np.nan
    else:
        # center = tuple([ l//2 for l in density.shape ])
        rho_d = density[center_slc].mean()
        rho_m = density[edge_slc].mean()
        # mass = np.sum(density - rho_m) + 0.5*Vp*np_sphere*rho_sphere
        mass = droplet_mass(density - rho_m) + 0.5*Vp*np_sphere*rho_sphere
        R = (3./4./np.pi*mass/(rho_d-rho_m))**(1./3.)
        return R

### `droplet_radius_iso`

**Input**
1. `density`: `np.ndarray` of shape `[nx, ny, nz]` which holds order parameter data of the droplet
2. `level`: `float` of isocontour to calculate radius from

In [ ]:
def droplet_radius_iso(density, level = None):
    if level is None:
        level = filters.threshold_otsu(density)
    verts, faces, normals, values = measure.marching_cubes(density, level = level)
    cm = center_of_mass(density)
    ri_s = np.linalg.norm(verts - cm, axis = 1)
    R = np.mean(ri_s)
    return R

### `inertia_tensor`

**Input**
1. `cm`: `np.ndarray` of size `3` which defines the center of mass of the droplet
2. `OutArray`: `np.ndarray` of order parameter data that has been pre-processed such that all other points are 0 except anything defining the droplet

In [ ]:
def inertia_tensor(cm,OutArray):
    ind = np.transpose(np.indices(OutArray.shape), axes=(1,2,3,0))
    pos = ind - cm
    r2 = np.einsum('ijkl,ijkl->ijk',pos,pos)          # inner product
    rr = np.einsum('ijkm,ijkn->ijkmn',pos,pos)        # outer product
    r2 = np.einsum('ijk,mn->ijkmn',r2,np.identity(3)) # multiply with unit matrix
    I = np.einsum('ijk,ijkmn->mn',OutArray,r2-rr)     # sum m*(r2-rr)
    return I

### `radii_pca`

**Input**
1. `eigvals`: `np.ndarray` of size `3` which defines the eigenvalues of the intertia tensor
2. `mass`: `float` describing the droplet mass

In [ ]:
def radii_pca(eigvals, mass):
    a = np.sqrt((5/(2*mass))*(eigvals[1]+eigvals[2]-eigvals[0]))
    b = np.sqrt((5/(2*mass))*(eigvals[0]+eigvals[2]-eigvals[1]))
    c = np.sqrt((5/(2*mass))*(eigvals[0]+eigvals[1]-eigvals[2]))

    return np.array([a, b, c])

### `gyration_tensor`

**Input**
1. `cm`: `np.ndarray` of size `3` which defines the center of mass of the droplet
2. `OutArray`: `np.ndarray` of order parameter data that has been pre-processed such that all other points are 0 except anything defining the droplet

In [ ]:
def gyration_tensor(cm,OutArray):
    ind = np.transpose(np.indices(OutArray.shape), axes=(1,2,3,0))
    pos = ind - cm
    rr = np.einsum('...m,...n->...mn',pos,pos)
    S = np.einsum('ijk,ijk...',OutArray,rr)/np.sum(OutArray)
    return S

### `droplet_fluctuations`

(fluctuations, temp = 1e-7)

**Input**
1. `fluctuations`: `np.ndarray` of size `[data_size, 3]` which defines the fluctuations of the principle radii of the droplet
2. `temp`: `float` of the temperature used

In [ ]:
def droplet_fluctuations(fluctuations, temp = 1e-7):
    sums = 0
    difs = 0

    for i in range(0, 2):
        for j in range(i+1, 3):
            sums += np.mean(np.power(fluctuations[:, i] + fluctuations[:, j], 2))
            difs += np.mean(np.power(fluctuations[:, i] - fluctuations[:, j], 2))

    sums *= 1/3
    difs *= 1/3

    y20 = 5*temp/(16*np.pi*sums)
    y22 = 15*temp/(16*np.pi*difs)

    return [y20, y22]

# Validation

## Homogeneous system

In [ ]:
L = 64
boxDims = [L, L, L]
use_hdf5 = False

chi = 0.4
T = 0.25
kappa = 0.01
kbt = 1e-7

rho0 = 1.0
phi0 = 0.0
thermo_vars = swift_et_al_1996_thermodynamic_model(rho0, phi0, chi, T, kappa)

# noise_type = "spatially_independent"
noise_type = "spatially_dependent"
save_dir = f"validation/equilibration_tests/{noise_type}"
# save_dir = "./"
binsize_avg = 16
binsize_radial = 16

freqs = fft.fftshift(fft.fftfreq(L))
if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
# if use_hdf5:
#     paths = sorted(glob.glob(f"{save_dir}/hydro*.h5"))[1:]

#     S_rho = np.zeros(([L]*3), dtype = complex)
#     S_phi = np.zeros([L]*3, dtype = complex)
#     S_v = np.zeros((*[L]*3,3,3), dtype = complex)

#     for i, h5_filepath in enumerate(paths):
#         conserved_moments = extract_data(h5_filepath, boxDims, begin = 0, end = 5, nVars = 38)

#         var = conserved_moments[0]
#         vark = fft.fftn(var)
#         S_rho += vark*np.conj(vark)

#         var = conserved_moments[1]
#         vark = fft.fftn(var)
#         S_phi += vark*np.conj(vark)

#         var = conserved_moments[2:]
#         vark = fft.fftn(var, axes = [1, 2, 3])
#         S_v += np.einsum('i..., j...->...ij',vark, np.conj(vark))


#     S_rho /= (len(paths)*L**3*kbt)
#     S_phi /= (len(paths)*L**3*kbt)
#     S_v /= (len(paths)*L**3*kbt)

#     data_rho = [S_rho.real]
#     data_phi = [S_phi.real]
#     data_v = [S_v[..., 0, 0].real, S_v[..., 1, 1].real, S_v[..., 2, 2].real]
# else:
#     ts = yt.load(f"{save_dir}/SF_plt_mag*")
#     ds = ts[-1]
#     ad = ds.all_data()

#     data_rho = [np.array(ad[('boxlib', 'struct_fact_density_density')]).reshape(boxDims)/kbt]
#     data_phi = [np.array(ad[('boxlib', 'struct_fact_phi_phi')]).reshape(boxDims)/kbt]
#     data_v = [np.array(ad[('boxlib', 'struct_fact_ux_ux')]).reshape(boxDims)/kbt, 
#               np.array(ad[('boxlib', 'struct_fact_uy_uy')]).reshape(boxDims)/kbt,
#               np.array(ad[('boxlib', 'struct_fact_uz_uz')]).reshape(boxDims)/kbt]

#     # ds.field_list

### Density

#### Spatially independent

In [ ]:
noise_type = "spatially_independent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_rho = [np.array(ad[('boxlib', 'struct_fact_density_density')]).reshape(boxDims)/kbt]

if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_rho
experiment_muck = thermo_vars.cs2k(kx, ky, kz)

test = data[0].copy()
# test /= kbt

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
test *= (experiment_muck)
# test[slc] /= test.sum()
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax, shrink = 0.8)

fig.tight_layout()

In [ ]:
slc = np.s_[L//2, L//2, L//2]

ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\rho$"]

rho_scale = lambda a, c: a*c
scale_factor = 1

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1, func = rho_scale)
        # y[0] = 1
        y[0] = np.NaN
        x, y = make_bins(x, binsize_avg, to_bin2 = y)
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim([0.95, 1.065])
fig.tight_layout()
fig.savefig(f"./{figures}/rho-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r, func=rho_scale, scale_factor = scale_factor)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", va = "bottom", ha = 'right', fontsize = 14, rotation = 'horizontal')
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig1.tight_layout()
fig2.tight_layout()
fig1.savefig(f"./{figures}/rho-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/rho-ER-{noise_type}-azimuth.png", dpi = 300)

#### Spatially dependent

In [ ]:
noise_type = "spatially_dependent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_rho = [np.array(ad[('boxlib', 'struct_fact_density_density')]).reshape(boxDims)/kbt]
if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_rho
experiment_muck = thermo_vars.cs2k(kx, ky, kz)

test = data[0].copy()
# test /= kbt

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
test *= (experiment_muck)
# test[slc] /= test.sum()
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax, shrink = 0.8)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax, shrink = 0.8)

fig.tight_layout()

In [ ]:
slc = np.s_[L//2, L//2, L//2]

ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\rho$"]

rho_scale = lambda a, c: a*c
scale_factor = 1

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1, func = rho_scale)
        # y[0] = 1
        y[0] = np.NaN
        x, y = make_bins(x, binsize_avg, to_bin2 = y)
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim([0.95, 1.05])
fig.tight_layout()
fig.savefig(f"./{figures}/rho-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 
binsize_temp = 16

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r, func=rho_scale, scale_factor = scale_factor)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \rho(k) | ^2 \rangle }{S(k)}$", va = "bottom", ha = 'right', fontsize = 14, rotation = 'horizontal')
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig1.tight_layout()
fig2.tight_layout()
fig1.savefig(f"./{figures}/rho-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/rho-ER-{noise_type}-azimuth.png", dpi = 300)

### C1

#### Spatially independent

In [ ]:
noise_type = "spatially_independent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_phi = [np.array(ad[('boxlib', 'struct_fact_phi_phi')]).reshape(boxDims)/kbt]
if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_phi

# freqs = fft.fftshift(fft.fftfreq(L))
# kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
# k2 = lattice_fourier_laplacian(kx, ky, kz)
experiment_muck = thermo_vars.mu_ck(kx, ky, kz)

test = data[0].copy()
# test[slc] = (test.sum() - test[slc])/(L**3)

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
# test[slc] = (test.sum() - test[slc])/32**3
test *= (experiment_muck)
# test /= kbt
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax)

fig.tight_layout()

In [ ]:
ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\phi$"]

rho_func = lambda a, c: a*c
scale_factor = 1

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = scale_factor, func = rho_func, cs = False)
        x, y = make_bins(x, binsize_avg, to_bin2 = y)

        # y[0] = 1
        y[0] = np.NaN
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
# print(x, y)
# ax.set_ylim([0.9, 1.1])
ax.set_ylim([0.95, 1.055])
fig.tight_layout()
fig.savefig(f"./{figures}/phi-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 
binsize_temp = 16

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor, cs = False)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", va = "bottom", ha = 'right', fontsize = 14, rotation = 'horizontal')
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig1.tight_layout()
fig2.tight_layout()
fig1.savefig(f"./{figures}/phi-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/phi-ER-{noise_type}-azimuth.png", dpi = 300)

#### Spatially dependent

In [ ]:
noise_type = "spatially_dependent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_phi = [np.array(ad[('boxlib', 'struct_fact_phi_phi')]).reshape(boxDims)/kbt]
if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_phi

# freqs = fft.fftshift(fft.fftfreq(L))
# kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
# k2 = lattice_fourier_laplacian(kx, ky, kz)
experiment_muck = thermo_vars.mu_ck(kx, ky, kz)

test = data[0].copy()
# test[slc] = (test.sum() - test[slc])/(L**3)

fig, axs = plt.subplots(1, 3, figsize = (9, 3))

ax = axs[0]
# test[slc] = (test.sum() - test[slc])/32**3
test *= (experiment_muck)
# test /= kbt
im = ax.imshow(test[L//2, :, :])
plt.colorbar(im, ax = ax)

ax = axs[1]
im = ax.imshow(test[:, L//2, :])
plt.colorbar(im, ax = ax)

ax = axs[2]
im = ax.imshow(test[:, :, L//2])
plt.colorbar(im, ax = ax)

fig.tight_layout()

In [ ]:
ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$\phi$"]

rho_func = lambda a, c: a*c
scale_factor = 1

for d in data:
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = scale_factor, func = rho_func, cs = False)
        x, y = make_bins(x, binsize_avg, to_bin2 = y)
        # y[0] = 1
        y[0] = np.NaN
        ax.plot(x, y, 
                marker = markers[0], color = colors[0],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[0])

ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
# print(x, y)
ax.set_ylim([0.95, 1.05])
fig.tight_layout()
fig.savefig(f"./{figures}/phi-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1.5, 2.5]

t_angle = 0
p_angle = 0 

# rho_scale = None
# scale_factor = 1/cs2

for d in data:
    ax1 = tax
    ax2 = pax
    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r, func=rho_scale, scale_factor = scale_factor, cs = False)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)


    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", fontsize = 14)
    ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | \phi(k) | ^2 \rangle }{S(k)}$", va = "bottom", ha = 'right', fontsize = 14, rotation = 'horizontal')
    ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

fig1.tight_layout()
fig2.tight_layout()
fig1.savefig(f"./{figures}/phi-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/phi-ER-{noise_type}-azimuth.png", dpi = 300)

### Velocities

#### Spatially independent

In [ ]:
noise_type = "spatially_independent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_v = [np.array(ad[('boxlib', 'struct_fact_ux_ux')]).reshape(boxDims)/kbt, 
          np.array(ad[('boxlib', 'struct_fact_uy_uy')]).reshape(boxDims)/kbt,
          np.array(ad[('boxlib', 'struct_fact_uz_uz')]).reshape(boxDims)/kbt]

if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_v
slc_s = [np.s_[L//2, :, :], np.s_[:, L//2, :], np.s_[:, :, L//2]]
labels = ['yz', 'xz', 'xy']
fig, axs = plt.subplots(3, 3, figsize = (9, 9))

for i in range(3):
    d = data[i].copy()
    # d /= kbt
    # d[slc] = (d.sum() - d[slc])/(L**3)
    for j in range(3):
        ax = axs[i, j]
        im = ax.imshow(d[slc_s[j]], vmin = 0.8, vmax = 1.2)
        plt.colorbar(im, ax = ax, shrink = 0.8)
        ax.set_title(f"$S_{{u_{chr(120+i)}u_{chr(120+i)}}}$ {labels[j]}")

fig.tight_layout()

In [ ]:
ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))
binsize = 8

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$u_x$", r"$u_y$", r"$u_z$"]

for i, d in enumerate(data):
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1)
        # y[0] = 1
        y[0] = np.NaN
        x, y = make_bins(x, binsize_avg, to_bin2 = y)

        ax.plot(x, y, 
                marker = markers[i], color = colors[i],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[i])
ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | u_{\alpha}(k) | ^2 \rangle }{\rho_{0}k_b T}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim([0.95, 1.05])

fig.tight_layout()
fig.savefig(f"./{figures}/vel-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
# fig1, tax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})
# fig2, pax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})
data = data_v[:1]
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1, 1.5]

t_angle = 0
p_angle = 0 

for j, d in enumerate(data):
    ax1 = tax
    ax2 = pax

    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | u_{x} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | u_{x} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14, ha = 'right', va = "top", rotation = 'horizontal')
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)

fig1.tight_layout()
fig2.tight_layout()

fig1.savefig(f"./{figures}/vel-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/vel-ER-{noise_type}-azimuth.png", dpi = 300)

#### Spatially dependent

In [ ]:
noise_type = "spatially_dependent"
save_dir = f"validation/equilibration_tests/{noise_type}"
ts = yt.load(f"{save_dir}/SF_plt_mag*")
ds = ts[-1]
ad = ds.all_data()

data_v = [np.array(ad[('boxlib', 'struct_fact_ux_ux')]).reshape(boxDims)/kbt, 
          np.array(ad[('boxlib', 'struct_fact_uy_uy')]).reshape(boxDims)/kbt,
          np.array(ad[('boxlib', 'struct_fact_uz_uz')]).reshape(boxDims)/kbt]

if "in" in noise_type:
    kx, ky, kz = [0, 0, 0]
else:
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')

In [ ]:
data = data_v
slc_s = [np.s_[L//2, :, :], np.s_[:, L//2, :], np.s_[:, :, L//2]]
labels = ['yz', 'xz', 'xy']
fig, axs = plt.subplots(3, 3, figsize = (9, 9))

for i in range(3):
    d = data[i].copy()
    # d /= kbt
    # d[slc] = (d.sum() - d[slc])/(L**3)
    for j in range(3):
        ax = axs[i, j]
        im = ax.imshow(d[slc_s[j]], vmin = 0.8, vmax = 1.2)
        plt.colorbar(im, ax = ax, shrink = 0.8)
        ax.set_title(f"$S_{{u_{chr(120+i)}u_{chr(120+i)}}}$ {labels[j]}")

fig.tight_layout()

In [ ]:
ar = 1.25
sz = figheight
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

colors = ['b', "r", "k"]
markers = ["o", "s", '^']
labels = [r"$u_x$", r"$u_y$", r"$u_z$"]

for i, d in enumerate(data):
        currData = d.copy()
        # currData /= kbt
        x, y = spherically_averaged_structure_factor(currData, thermo_vars, scale_factor = 1)
        # y[0] = 1
        y[0] = np.NaN        
        x, y = make_bins(x, binsize_avg, to_bin2 = y)

        ax.plot(x, y, 
                marker = markers[i], color = colors[i],
                markerfacecolor = "None", markersize = 10, 
                linestyle = "None", label = labels[i])
ax.plot(x, [1]*x.size, 'lime', lw = 2)

ax.set_xlabel(r"$k$", fontsize = 14)
ax.set_ylabel(r"$\frac{ \langle | u_{\alpha}(k) | ^2 \rangle }{\rho_{0}k_b T}$", fontsize = 14)

ax.tick_params(axis='both', which='major', labelsize=12)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.legend()
ax.set_ylim([0.95, 1.05])

fig.tight_layout()
fig.savefig(f"./{figures}/vel-ER-{noise_type}-avg.png", dpi = 300)

In [ ]:
sz = figheight
# fig1, tax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})
# fig2, pax = plt.subplots(1, 3, figsize = (sz*3, sz), subplot_kw={'projection': 'polar'})
data = data_v[:1]
fig1, tax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})
fig2, pax = plt.subplots(1, 1, figsize = (sz, sz), subplot_kw={'projection': 'polar'})

colors = ['b', "r", "k"]
markers = ["o", "s", '^']

radii = [0.5, 1, 1.5]

t_angle = 0
p_angle = 0 

for j, d in enumerate(data):
    ax1 = tax
    ax2 = pax

    currData = d.copy()
    # currData /= kbt
    for i, r in enumerate(radii):
        # t, p, sf = radial_equilibration(d, rho0, phi0, radius = r)
        t, p, sf = radial_equilibration(currData, thermo_vars, radius = r)

        idxs = np.isclose(np.abs(t), t_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(p)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax1.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

        idxs = np.isclose(np.abs(p), p_angle/180*np.pi, atol = np.pi/L)
        angles = np.abs(t)[idxs]
        segment = sf[idxs]
        angles, segment =  make_bins(angles, binsize_radial, to_bin2 = segment)
        ax2.plot(angles, segment, color = colors[i], label = f"R = {r}",lw = 2)

    ax1.set_rticks([0.5, 1.0])
    ax1.set_thetalim([0, np.pi/2])
    ax1.set_ylabel(r"$\frac{ \langle | u_{x} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14)
    # ax1.set_title(r"$\theta = {{{0}}}^{{\circ}}$".format(t_angle))

    ax2.set_rticks([0.5, 1.0])
    ax2.set_thetalim([0, np.pi])
    ax2.set_ylabel(r"$\frac{ \langle | u_{x} | ^2 \rangle }{\rho_0 k_B T}$", fontsize = 14, ha = 'right', va = "top", rotation = 'horizontal')
    # ax2.set_title(r"$\psi = {{{0}}}^{{\circ}}$".format(p_angle))

ax1.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)
ax2.legend(loc = 'upper right', bbox_to_anchor=(1.1, 1.1), fancybox=True)

fig1.tight_layout()
fig2.tight_layout()

fig1.savefig(f"./{figures}/vel-ER-{noise_type}-polar.png", dpi = 300)
fig2.savefig(f"./{figures}/vel-ER-{noise_type}-azimuth.png", dpi = 300)

### Figure compositing

In [ ]:
# independent composited figure
# %%capture
fig, axs = plt.subplots(2, 3, figsize=(3*figheight, 2*figheight))

noise_type = "spatially_independent"
ordering = ['rho', 'phi', 'vel']

downsize = 2
for i, ax in enumerate(axs[0]):
    pic_path = f"{figures}/{ordering[i]}-ER-{noise_type}-avg.png"
    img = iio.imread(pic_path)
    if downsize is not None:
        img = ski.transform.resize(img, (img.shape[0]//downsize, img.shape[1]//downsize), anti_aliasing=True)
    
    ax.imshow(img)
    ax.axis('off')
    ax.text(0.0, 0.9, f"({chr(97+i)})", transform=ax.transAxes)

paths = [f"{figures}/rho-ER-{noise_type}-polar.png", f"{figures}/phi-ER-{noise_type}-polar.png", f"{figures}/vel-ER-{noise_type}-polar.png"]

for i, ax in enumerate(axs[1]):
    pic_path = paths[i]
    # print(pic_path)
    img = iio.imread(pic_path)
    if downsize is not None:
        img = ski.transform.resize(img, (img.shape[0]//downsize, img.shape[1]//downsize), anti_aliasing=True)
    
    ax.imshow(img)
    ax.axis('off')
    ax.text(0.0, 0.9, f"({chr(97+3+i)})", transform=ax.transAxes)

fig.subplots_adjust(wspace=-0.1, hspace=0)
fig.tight_layout()
fig.savefig(f"{figures}/{noise_type}-summary.png", dpi=DPI)#, bbox_inches='tight')

In [ ]:
# dependent composited figure
# %%capture
fig, axs = plt.subplots(2, 3, figsize=(3*figheight, 2*figheight))

noise_type = "spatially_dependent"
ordering = ['rho', 'phi', 'vel']

downsize = None
for i, ax in enumerate(axs[0]):
    pic_path = f"{figures}/{ordering[i]}-ER-{noise_type}-avg.png"
    img = iio.imread(pic_path)
    if downsize is not None:
        img = ski.transform.resize(img, (img.shape[0]//downsize, img.shape[1]//downsize), anti_aliasing=True)
    
    ax.imshow(img)
    ax.axis('off')
    ax.text(0.0, 0.9, f"({chr(97+i)})", transform=ax.transAxes)

paths = [f"{figures}/rho-ER-{noise_type}-polar.png", f"{figures}/phi-ER-{noise_type}-polar.png", f"{figures}/vel-ER-{noise_type}-polar.png"]

for i, ax in enumerate(axs[1]):
    pic_path = paths[i]
    # print(pic_path)
    img = iio.imread(pic_path)
    if downsize is not None:
        img = ski.transform.resize(img, (img.shape[0]//downsize, img.shape[1]//downsize), anti_aliasing=True)
    
    ax.imshow(img)
    ax.axis('off')
    ax.text(0.0, 0.9, f"({chr(97+3+i)})", transform=ax.transAxes)

fig.subplots_adjust(wspace=-0.1, hspace=0)
fig.tight_layout()
fig.savefig(f"{figures}/{noise_type}-summary.png", dpi=DPI)#, bbox_inches='tight')

## Interfacial fluctuations

In [ ]:
nx = 256
ny = 1
nz = 2048
noise_type = "spatially_independent"
# noise_type = "spatially_dependent"
savedir = f"./validation/interface/{noise_type}"

# nx = 64
# ny = 2
# nz = 512
# savedir = "./"

boxDim = np.array([nx, ny, nz])

kappa = 0.03
kbt = 1e-7
u0 = 0
cs2 = 1/3
gamma = 1.0

chi = 0.5
T = 0.2

idx = -1

output_file = "hydro_plt"

In [ ]:
boxDim = np.array([nx, ny, nz])

# profile = extract_data(savedir + sorted(glob.glob("./*.h5"))[-1], [nx, ny, nz], begin = 1, end = 2, nVars = 38)
path = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[idx]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

# ts = yt.load(f"{savedir}/{output_file}*")
# ds = ts[-1]
# ad = ds.all_data()
# profile = np.array([np.array(ad[('boxlib', 'density')]).reshape(boxDim), np.array(ad[('boxlib', 'phi')]).reshape(boxDim)])

phi0 = newton(fb, x0 = (0.6), args = (chi, T))
xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
# phi0 = np.sqrt((3*(chi/2 - T))/(2*T))
# xi = 4*np.sqrt(kappa*(chi/2 - T))
fit_func = lambda x, b:phi0*np.tanh((x - b)/(np.sqrt(2)*xi))

fig, axs = plt.subplots(2, 2, figsize = (8, 8))
axs = axs.flatten()

ax = axs[0]
im = ax.imshow(profile[0, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\rho$")

ax = axs[1]
x = np.arange(nx//4, 3*nx//4, 1)
im = ax.plot(x, profile[0, nx//4:3*nx//4, ny//2 , nz//2], 'rx', label = "profile")
# ax.plot(x, np.ones(x.size), 'ko', label = "reference", markerfacecolor="None")

ax.set_xlabel("x")
ax.set_ylabel(r"$\rho$")
ax.legend(ncol = 1, fontsize = 'small')

ax = axs[2]
im = ax.imshow(profile[1, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\phi$")

ax = axs[3]
x = np.arange(nx//4, 3*nx//4, 1)
im = ax.plot(x, profile[1, nx//4:3*nx//4, ny//2 , nz//2], 'rx', label = "profile")

x = np.linspace(0, nx - 1, nx)
# b = -(chi**2.350)/(kappa**0.640 * T**5.559*489.435)
# b = 
# y = newton(fb, x0 = (0.6), args = (chi, T))*np.tanh(b*(x - (nx - 1)/2))
y = fit_func(x, (nx - 1)/2)
ax.plot(x, y, 'ko', label = "referece", markerfacecolor="None")

ax.set_xlabel("x")
ax.set_ylabel(r"$\phi$")
ax.legend(ncol = 1, fontsize = 'small')

t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
fig.suptitle(f"$\kappa$ = {kappa}, $\chi = {chi}, T = {T}, t = {t}$")
fig.tight_layout()

In [ ]:
xlo = nx//4
xhi = 3*nx//4
intLoc = (nx - 1)/2

yraw = profile[1, :, 0, nz//2]

## Raw data ##
plt.plot(yraw, 'rx', label = 'raw data')
## Raw data ##

## tanh fit ##
phi0 = newton(fb, x0 = (0.6), args = (chi, T))
xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
# phi0 = np.sqrt((3*(chi/2 - T))/(2*T))
# xi = 4*np.sqrt(np.sqrt(2)*kappa*(chi/2 - T))

xfit1 = np.arange(xlo, xhi, 1)
fit_func = lambda x, b:phi0*np.tanh((x - b)/(np.sqrt(2)*xi))
popt, pcov = curve_fit(fit_func, xfit1, yraw[xlo:xhi], p0 = (intLoc))
yfit1 = fit_func(xfit1, *popt)
intHeight_prof = popt[0]
plt.plot(xfit1, yfit1, 'b--', label = "tanh fit", lw = 0.6)
plt.plot(intHeight_prof, 0, 'bo', ms = 8)
## tanh fit ##

## direct fit ##
xslc = [int(np.floor(intLoc)), int(np.ceil(intLoc))]
yslc = [yraw[xslc[0]], yraw[xslc[1]]]
plt.plot(xslc, yslc, linestyle = '-', marker = "None", color = 'tab:orange', label = 'direct fit', lw = 1)
intHeight_dir = np.roots(np.polyfit(xslc, yslc, 1))[0]
plt.plot(intHeight_dir, 0, marker = "o", color = "tab:orange", ms = 8)
## direct fit ##

plt.ylabel(r"$\phi$")
plt.xlabel(r"$x$")
plt.title("Fluctuations on")
plt.legend()
plt.xlim([123, 133])

print(f"profile fit:{intHeight_prof:.3f}, direct fit:{intHeight_dir:.3f}")

In [ ]:
path = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[1]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

methods = ["direct", "profile_fit"]

fig, ax = plt.subplots(1, 1, figsize = (5, 4))

colors = ['tab:orange', 'black']
ls = []

for i, method in enumerate(methods):
    h = interface_height(profile[1], chi, T, kappa, method = method)
    h = np.mean(h, axis = 0)
    ax.plot(h, label = method, color = colors[i])
    ls.append(h)

ax.set_ylabel("Height fluctuations")
ax.legend()

In [ ]:
method = "direct"
fft_mode = 'ortho'

# method = "profile_fit"
# fft_mode = 'ortho'

heights_k = np.zeros((nz), dtype = np.complex128)
# h5_paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[102:]
data_paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[1:102]

# data_paths = yt.load(f"{savedir}/{output_file}*")[1:]
# ds = ts[-1]
# ad = ds.all_data()
# profile = np.array([np.array(ad[('boxlib', 'density')]).reshape(boxDim), np.array(ad[('boxlib', 'phi')]).reshape(boxDim)])


for path in data_paths:
    phi = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)[0]
    # ad = path.all_data()
    # phi = np.array(ad[('boxlib', 'phi')]).reshape(boxDim)
    
    h = interface_height(phi, chi, T, kappa, method = method)[0]*np.sqrt(2)
    
    h_k = fft.fft(h, norm = fft_mode)

    heights_k += h_k*h_k.conjugate()
    # t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
    # print(f"Timestep {t} processed")

heights_k = np.abs(heights_k)
heights_k /= len(data_paths)

In [ ]:
## EXPERIMENTAL RESULTS ##
L = nz
k1 = fft.fftfreq(L)*2*np.pi
S1 = heights_k.copy()

slc = slice(1, L//2)
xraw = k1[slc]
yraw = S1[slc]

binsize = L//4
kmin = 2*np.pi/binsize
bins = np.arange(binsize//2+1)*kmin # kmax+1 for bin_edges: len(bins)=len(hist)+1
shells = np.histogram(k1[slc], bins, weights=S1[slc])[0]
counts = np.histogram(k1[slc], bins)[0]

xbin = (bins[:-1]+bins[1:])/2
ybin = shells/counts
## EXPERIMENTAL RESULTS ##

## THEORETICAL RESULTS ##
sigma = 0.0172
freqs_theory = np.linspace(xraw[0], xraw[-1], 1001)
interface_fluct_theory = kbt/(sigma*np.power(freqs_theory, 2))
## THEORETICAL RESULTS ##

sz = figheight
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

# ax.loglog(xraw, yraw, "ks", label = "Simulation raw", ms = 5, markerfacecolor = "None")
ax.loglog(xbin, ybin, "bo", label = "Simulation", ms = 3)
ax.loglog(freqs_theory, interface_fluct_theory, 'r-', lw = 2, label = "Theory")

ax.set_xlim(left = 1e-2)
ax.set_xlabel(r"$k$", fontsize = 15)
ax.set_ylabel(r"$\langle |h(k)|^2 \rangle$", fontsize = 15)
ax.tick_params(axis='both', which='major', labelsize=12)

ax.legend(fontsize = 12, loc = 'lower left')
# # ax.set_title(f"L = {nz}")
# ax.set_title(f"{method}")

fig.tight_layout()
fig.savefig(f"{figures}/interface_height_flucuations-{noise_type}.png", dpi = 300)

## Droplet fluctuations

Sources
1. [Simulation of FUS Protein Condensates with an adapted coarse grained model](https://pubs.acs.org/doi/10.1021/acs.jctc.0c01064)
2. [Effect of nanoparticles and surfactants on droplets in shear flows](https://doi.org/10.1039/C2SM25209K)

### Young Laplace fit

In [ ]:
nx = 64
ny = 64
nz = 64

boxDim = np.array([nx, ny, nz])

chi = 0.5
T = 0.2
kappa = 0.03
kbt = 1e-7

u0 = 0
cs2 = 1/3
gamma = 1.0

idx = -1
savedir = f"./validation/droplet_fluctuations/young_laplace"
# R_s = ["0.2", "0.25", "0.3", "0.35"]
R_s = ["0.25", "0.3", "0.35"]
output_file = "hydro_plt"

In [ ]:
cutoff = 0

all_radii = []
all_times = []
all_press = []

for R in R_s:
    paths = sorted(glob.glob(f"{savedir}/R_{R}/{output_file}*.h5"))

    curr_radii = np.zeros(len(paths))    
    curr_times = np.zeros(len(paths))
    curr_press = np.zeros(len(paths))
    for i, path in enumerate(paths):
        t = int(path.split("_")[-1].split(".")[0])    
        curr_times[i] = t

        profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

        curr_radii[i] = droplet_radius_mass(profile[1])

        pressure = pressure_tensor(profile, T, kappa)
        scalar_pressure = np.einsum('ijkmn,mn->ijk',pressure,np.identity(boxDim.size))/3
        curr_press[i] = pressure_jump(scalar_pressure)
        
    
    all_radii.append(curr_radii)
    all_times.append(curr_times)
    all_press.append(curr_press)

In [ ]:
sz = 4
ar = 1.25
fig, axs = plt.subplots(1, 2, figsize = (sz*ar*2, sz))
axs2 = [ax.twinx() for ax in axs]

cols = ['r', 'b', 'k', 'g']
markers = ['s', 'o', '^', '*']
linestyles = ['-', '--', ':', 'dashdot']

for i, R in enumerate(R_s):
    x = all_times[i]
    y = all_radii[i]
    y2 = all_press[i]
    if len(x) > 1:
        plotx = x[1:]
        ploty = y[1:]
        ploty2 = y2[1:]

        ax = axs[0]
        ax.plot(plotx, ploty, label = f'R = {R}', marker = markers[i], linestyle = "None", color = cols[i], markerfacecolor = "None", ms = 8)

        ax2 = axs2[0]
        ax2.plot(plotx, ploty2, label = f'R = {R}', marker = "None", linestyle = linestyles[i], color = cols[i], markerfacecolor = "None", ms = 8)

        ax = axs[1]
        ploty = (ploty - ploty.min())/(ploty.max() - ploty.min())
        ax.plot(plotx, ploty, label = f'R = {R}', marker = markers[i], linestyle = "None", color = cols[i], markerfacecolor = "None", ms = 8)

        ax2 = axs2[1]
        ploty = (ploty2 - ploty2.min())/(ploty2.max() - ploty2.min())
        ax2.plot(plotx, ploty, label = f'R = {R}', marker = "None", linestyle = linestyles[i], color = cols[i], markerfacecolor = "None", ms = 8)
    
ax = axs[0]

ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$R$")
ax.set_xscale("log")
# ax.legend(loc = 'center right')

ax2 = axs2[0]
ax2.set_ylabel(r"$\Delta P$")

ax = axs[1]
ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$\frac{R - R_{min}}{R_{max} - R_{min}}$")
ax.set_xscale("log")
# ax.legend(loc = 'center right')
ax.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))

ax2 = axs2[1]
ax2.set_ylabel(r"$\frac{\Delta P - \Delta P_{min}}{\Delta P_{max} - \Delta P_{min}}$")


fig.suptitle("Droplet radius evolution over time")
fig.tight_layout()

In [ ]:
young_laplace = lambda x, sigma: sigma*x

xraw = []
yraw = []
for i in range(len(R_s)):
    ls = all_radii[i]
    if len(ls) > 1:
        xraw.append(2/all_radii[i][-1])
        yraw.append(all_press[i][-1])

sz = figheight
ar = 1.25
fig, ax = plt.subplots(1, 1, figsize = (sz*ar, sz))

ax.plot(xraw, yraw, 'bo', ms = 8, linestyle = "None", label = "Raw data", markerfacecolor = "None")

popt, pcov = curve_fit(young_laplace, xraw, yraw, p0 = [0.0170])
xfit = np.linspace(min(xraw), max(xraw), 101)
yfit = young_laplace(xfit, *popt)
ax.plot(xfit, yfit, 'r-', lw = 1, label = "Fit data")

ax.text(0.12, 0.002, r"$\sigma$="+f"{popt[0]:.4f}", fontsize = 12)
ax.set_ylabel(r"$\Delta P$", fontsize = 15)
ax.set_xlabel(r"$\frac{2}{R_d}$", fontsize = 15)
# ax.set_title(f"$\chi$={chi}, $T$={T}, $\kappa$={kappa}", fontsize = 18)
# ax.legend(fontsize = 12)

fig.tight_layout()
fig.savefig(f"{figures}/young_laplace_sigma.png", dpi = 300)

### Fluctuations of droplet shape

In [ ]:
nx = 64
ny = 64
nz = 64

boxDim = np.array([nx, ny, nz])

kappa = 0.03
kbt = 1e-7
u0 = 0
cs2 = 1/3
gamma = 1.0

chi = 0.5
T = 0.2

idx = 50

# noise_type = "spatially_dependent"
noise_type = "spatially_independent"
savedir = f"./validation/droplet_fluctuations/{noise_type}"
output_file = "hydro_plt"

In [ ]:
boxDim = np.array([nx, ny, nz])

path = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[idx]
profile = extract_data(path, boxDim, begin = 0, end = 2, nVars = 38)

fig, axs = plt.subplots(2, 2, figsize = (8, 8))
axs = axs.flatten()

ax = axs[0]
im = ax.imshow(profile[0, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\rho$")

ax = axs[1]
x = np.arange(0, nx, 1)
im = ax.plot(x, profile[0, :, ny//2 , nz//2], 'rx', label = "profile")
ax.set_xlabel("x")
ax.set_ylabel(r"$\rho$")
ax.legend(ncol = 1, fontsize = 'small')

ax = axs[2]
im = ax.imshow(profile[1, :, : , nz//2])
ax.set_xlabel("y")
ax.set_ylabel("x")
plt.colorbar(im, ax = ax, label = r"$\phi$")

ax = axs[3]
x = np.arange(0, nx, 1)
im = ax.plot(x, profile[1, :, ny//2 , nz//2], 'rx', label = "profile")
ax.set_xlabel("x")
ax.set_ylabel(r"$\phi$")
ax.legend(ncol = 1, fontsize = 'small')

t = int(path.split("/")[-1].split(".")[0].split("_")[-1])
fig.suptitle(f"$\kappa$ = {kappa}, $\chi = {chi}, T = {T}, t = {t}$")
fig.tight_layout()

In [ ]:
# from skimage import filters

paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))

profile = extract_data(paths[50], boxDim, begin = 0, end = 2, nVars = 38)
field = profile[1].copy()

fig, axs = plt.subplots(1, 3, figsize = (9, 3))
colors = ['lime', 'magenta', 'blue', 'orange', 'red', 'k']
cutoffs = [filters.threshold_otsu(field, nbins = 2**4), filters.threshold_minimum(field),
           filters.threshold_li(field, initial_guess = 0), filters.threshold_mean(field),
           filters.threshold_triangle(field), filters.threshold_yen(field, nbins = 2**4)]

labels = ['otsu', 'min', 'li', 'mean', 'triangle', 'yen']

y_slc = 15 # edit me to change which slice of data we are looking at
slc_2D = np.s_[:, y_slc, :]
slc_1D = np.s_[:, y_slc, nz//2]

ax = axs[0]
ax.hist(field.ravel(), bins=2**4, density = True)
for i in range(len(colors)):
    ax.axvline(cutoffs[i], color=colors[i], label = labels[i])

ax.set_title("Comparing thresholding techniques")
ax.legend(ncol = 1)

ax = axs[1]
# field = np.where(field > cutoff, field, -0.03)
im = ax.imshow(field[slc_2D])
plt.colorbar(im, ax = ax)
ax.plot(np.arange(0, nz, 1), np.ones(nz)*ny//2, 'r-', lw = 1, label = 'slice')
ax.set_title("2D vis of data\n after mask application")
ax.legend()


ax = axs[2]
ax.plot(field[slc_1D], color = 'bisque', label = 'raw', markerfacecolor = "None", marker = 'o', linestyle = "None")

for i in range(len(colors)):
    ax.plot(np.arange(0, nz, 1), np.ones(nz)*cutoffs[i], color=colors[i], label = labels[i])
ax.set_title("Comparing slice of data\n and mask position")

ax.legend()

# cm = center_of_mass(field)
# gr = gyration_tensor(cm, field)
# egr = np.linalg.eigvals(gr)

# a = R_ref*np.power(egr[0], 1/3)/np.power(np.prod(egr[1:]), 1/6) - R_ref
# b = R_ref*np.power(egr[1], 1/3)/np.power(np.prod(egr[[0, 2]]), 1/6) - R_ref
# c = R_ref*np.power(egr[2], 1/3)/np.power(np.prod(egr[0:2]), 1/6) - R_ref

fig.tight_layout()

In [ ]:
# fig, ax = filters.try_all_threshold(profile[1][32], figsize=(10, 6), verbose=False)

In [ ]:
# rho = phi = profile[0]
# phi = profile[1]

# print(f"center phi:{phi[nx//2, ny//2, nz//2]}, edge:phi:{phi[0, 0, 0]}")

# L, L, L = phi.shape
# dims = [L, L, L]
# slc = np.s_[:L, :L, L//2]

# ## Convolving FFT with kernel ##
# kernel = np.ones(dims)
# rhok = fft.fftn(rho)
# phik = fft.fftn(phi)
# mode = "full"
# # mode = "same"
# rho_convolve = scipy.signal.convolve(rhok, kernel, mode = mode)/np.prod(dims)
# phi_convolve = scipy.signal.convolve(phik, kernel, mode = mode)/np.prod(dims)
# ## Convolving FFT with kernel ##

# ## Multiplying convolution with its adjoint ##
# complex_adjoint = rho_convolve.conj()
# rho_convolve = rho_convolve*complex_adjoint
# complex_adjoint = phi_convolve.conj()
# phi_convolve = phi_convolve*complex_adjoint
# ## Multiplying convolution with its adjoint ##

# sz = 4
# ar = 1
# row = 2
# col = 4
# fig = plt.figure(1, figsize=(sz*col*ar, sz*row))
# # numbers are in row-col-figure with figure number being row major

# fig.add_subplot(241)
# plt.imshow(rho[slc])
# plt.colorbar()
# plt.title(r"$\rho(r)$")

# fig.add_subplot(245)
# plt.imshow(phi[slc])
# plt.colorbar()
# plt.title(r"$\phi(r)$")

# fig.add_subplot(242)
# plt.imshow(rhok.real[slc])
# plt.colorbar()
# plt.title(r"$\rho(k)$")

# fig.add_subplot(246)
# plt.imshow(phik.real[slc])
# plt.colorbar()
# plt.title(r"$\phi(k)$")

# fig.add_subplot(243)
# plt.imshow(rho_convolve[slc].real, vmin = 1, vmax = 1)
# plt.colorbar()
# plt.title(r"$real(\rho_k^{conv})$")

# fig.add_subplot(247)
# plt.imshow(phi_convolve[slc].real, vmin = phi_convolve.real[slc].min(), vmax = phi_convolve.real[slc].max())
# plt.colorbar()
# plt.title(r"$real(\phi_k^{conv})$")

# fig.add_subplot(244)
# plt.imshow(rho_convolve[slc].imag, vmin = 0, vmax = 0)
# plt.colorbar()
# plt.title(r"$imag(\rho_k^{conv}$)")

# fig.add_subplot(248)
# plt.imshow(phi_convolve[slc].imag, vmin = 0, vmax = 0)
# plt.colorbar()
# plt.title(r"$imag(\phi_k^{conv}$)")

# fig.tight_layout()
# print(rho_convolve.shape)

In [ ]:
paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))[:51]
 
R_s = np.zeros(len(paths))
times = np.zeros(len(paths))

for i, path in enumerate(paths):
    t = int(path.split("_")[-1].split(".")[0])    
    times[i] = t
    
    profile = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)[0]
    field = profile.copy()

    R_s[i] = droplet_radius_mass(field)
    

fig, ax = plt.subplots(1, 1, figsize = (4, 3))

ls = []
ln, = ax.plot(times[1:], R_s[1:], 'g*', markerfacecolor = "None", ms = 8, label = 'R')
ls.append(ln)

print(R_s[-1])


ax.set_xlabel("Timesteps")
ax.set_ylabel(r"$R_{mass}$")
ax.set_title(f"t = {t}")

ax.legend(handles = ls)
ax.set_xscale("log")

fig.tight_layout()
fig.savefig(f"./{figures}/droplet_radius.png", dpi = 300)

In [ ]:
method = 'gyration'
# method = 'inertia'

paths = sorted(glob.glob(f"{savedir}/{output_file}*.h5"))
slc = slice(50, len(paths))
paths = paths[slc]
profile = extract_data(paths[0], boxDim, begin = 0, end = 2, nVars = 38)
field = profile[1].copy()
# cutoff = filters.threshold_otsu(field, nbins = 2**4)
cutoff = 0

if method == 'gyration':
    # GYRATION TENSOR METHOD ##
    R_ref = droplet_radius_iso(field)
    print(R_ref, cutoff)
    # GYRATION TENSOR METHOD ##
elif method == 'inertia':
    # INERTIA TENSOR METHOD ##
    field = np.where(field >= cutoff, field, 0)
    I_T = inertia_tensor(center_of_mass(field), field)
    R_ref = radii_pca(np.linalg.eigvals(I_T), droplet_mass(field))
    print(R_ref, cutoff)
    # INERTIA TENSOR METHOD ##

paths = paths[1:]
times = np.zeros(len(paths))
radiis = np.zeros((len(paths), 3))

for i, path in enumerate(paths):
    t = int(path.split("_")[-1].split(".")[0])    
    times[i] = t

    profile = extract_data(path, boxDim, begin = 1, end = 2, nVars = 38)[0]
    field = profile.copy()
    field = np.where(field >= cutoff, field, 0)
    cm = center_of_mass(field)
    
    if method == 'gyration':
        # GYRATION TENSOR METHOD ##
        # R_ref = droplet_radius_iso(field)
        gr = gyration_tensor(cm, field)
        egr = np.linalg.eigvals(gr)
        
        da = R_ref*np.power(egr[0], 1/3)/np.power(np.prod(egr[[1,2]]), 1/6) - R_ref
        db = R_ref*np.power(egr[1], 1/3)/np.power(np.prod(egr[[0,2]]), 1/6) - R_ref
        dc = R_ref*np.power(egr[2], 1/3)/np.power(np.prod(egr[[0,1]]), 1/6) - R_ref

        radiis[i] = [da,db,dc]
        # GYRATION TENSOR METHOD ##
    elif method == 'inertia':
        # INERTIA TENSOR METHOD ##
        I_T = inertia_tensor(cm, field)
        eit = np.linalg.eigvals(I_T)
        mass = droplet_mass(field)
        R = radii_pca(eit, mass)
        radiis[i] = R - R_ref
        # INERTIA TENSOR METHOD ##

print(np.mean(radiis, axis = 0))

In [ ]:
def droplet_fluctuations(fluctuations, temp = 1e-7):
    sums = 0
    difs = 0

    for i in range(0, 2):
        for j in range(i+1, 3):
            sums += np.mean(np.power(fluctuations[:, i] + fluctuations[:, j], 2))
            difs += np.mean(np.power(fluctuations[:, i] - fluctuations[:, j], 2))

    sums *= 1/3
    difs *= 1/3

    print(sums, difs)

    # y20 = 15*temp/(16*np.pi*sums)
    # y22 = 45*temp/(16*np.pi*difs)

    y20 = 5*temp/(16*np.pi*sums)
    y22 = 15*temp/(16*np.pi*difs)

    return [y20, y22]

In [ ]:
# def droplet_fluctuations(fluctuations, temp = 1e-7):
#     sz, dims = fluctuations.shape
#     sums = np.zeros(sz)
#     difs = np.zeros(sz)

#     for i in range(sz):
#         curr_sum = 0
#         curr_dif = 0
#         for j in range(0, dims - 1):
#             for k in range(j + 1, dims):
#                 curr_sum += np.power(fluctuations[i, j] + fluctuations[i, k], 2)
#                 curr_dif += np.power(fluctuations[i, j] - fluctuations[i, k], 2)

#         sums[i] = curr_sum/dims
#         difs[i] = curr_dif/dims

#     sums = sums.mean()
#     difs = difs.mean()

#     print(sums, difs)

#     # y20 = 15*temp/(16*np.pi*sums)
#     # y22 = 45*temp/(16*np.pi*difs)

#     y20 = 5*temp/(16*np.pi*sums)
#     y22 = 15*temp/(16*np.pi*difs)

#     return [y20, y22]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (4, 3))

marker_cols = ['rx', 'bo', 'ks']
labels = [r"$\delta R_{x}$", r"$\delta R_{y}$", r"$\delta R_{z}$"]

slc_pca = slice(50, radiis.shape[0])

for i in range(3):
    ax.plot(times[slc_pca], radiis[slc_pca, i], marker_cols[i], label = labels[i], markerfacecolor = 'none')

ax.set_xlabel("Timesteps")
# ax.set_ylabel(r"$R_{PCA} - R_{ref}$")
ax.set_ylabel(r"$\delta R_{\alpha}$")
ax.set_title(f"{method}")
ax.set_ylim(top = radiis[slc_pca].max() + 0.0015)
ax.legend(ncol = 3)

sigma_fluct = droplet_fluctuations(radiis[slc_pca], temp = kbt) # y20 y22
print(r"$\sigma_{20}$="+f"{sigma_fluct[0]:.4f}" + r" $\sigma_{22}$="+f"{sigma_fluct[1]:.4f}" + r" $\sigma_{YL}$=0.017")
print(np.mean(sigma_fluct))

fig.tight_layout()
# fig.savefig(f"./{figures}/droplet_shape_fluctuations-{noise_type}-avg.png", dpi = 300)

# Valid variable range

From the covariance matrix, certain limits on parameter values are set based upon ensuring that the diagonal remains positive. This limit is set by the term. $5 - c_s^2(k)$ in Xi[5, 5] which corresponds to a maximum allowable $c_s^2(k) = 0.\bar{5}$. The parameter that controls $c_s^2$ is $T$ as $c_s^2 = T$. $T_c$ or the critical temperature where demixing begins is defined as $T_c = \lambda/2$. In the expression for calculating $c_s^2(k) = c_s^2 + \kappa \rho_0 k^2$, $\kappa$ also controls the value of $c_s^2(k)$. Therefore this phase diagram will be defined using $\lambda$ and $\kappa$. Tested ranges will be $0.1 \leq \lambda \leq 1.1$ and $0.01 \leq \kappa \leq 0.05$

#### Helper functions

In [ ]:
@njit
# Newton-Raphson method function
def newton_c_ver(initial_guess, chi = 1.1, T = 0.5, tolerance = 1e-4, max_iterations = 1000):
    x = initial_guess  # value of phi
    rho = 1.0

    for i in range(max_iterations):
        fx = calculate_df_dphi(rho, x, chi, T)
        dfx = calculate_dmup_dphi(rho, x, chi, T)

        # Prevent division by zero
        if dfx == 0.0:
            # print("Divide by 0 encountered in Newton Raphson")
            # sys.exit(-1)
            return None

        x_next = x - fx / dfx

        # Check if the difference between successive iterations is within tolerance
        if abs(x_next - x) < tolerance:
            return x_next

        x = x_next

    # print("Maximum iteration reached without convergence in Newton Raphson")
    return None

@njit()
def lattice_fourier_laplacian(kx, ky, kz):
    expr1 = np.cos(kx) + np.cos(ky) + np.cos(kz)
    expr2 = np.cos(kx)*np.cos(ky) + np.cos(ky)*np.cos(kz) + np.cos(kx)*np.cos(kz)
    out = 2/9*expr1 + 2/9*expr2 - 4/3
    return -out/(1/3)

@njit()
def calculate_dmup_drho(rho, phi, chi, T):
    out = -T*phi/(rho**2 - phi**2) + chi/2*phi/(rho**2)
    return out

@njit()
def calculate_dmup_dphi(rho, phi, chi, T):
    out = T*rho/(rho**2 - phi**2) - chi/(2*rho)
    return out

@njit()
def sound_speed_square(T):
    out = T
    return out

@njit()
def calculate_df_dphi(rho, phi, chi, T):
    out = -chi/2.*(phi/rho) + T/2.*np.log((1. + phi/rho)/(1. - phi/rho))
    return out

@njit()
def cholesky_decomp(arr_in, n, bstart):
    A = arr_in.copy()
    # sum = 0
    for i in range(bstart, n):
        for j in range(bstart, i + 1):
            sum = A[i*n + j]
            for k in range(j - 1, bstart - 1, -1):
                sum -= A[i*n+k]*A[j*n+k]
            if i == j:
                if sum >= 0:
                    A[i*n+j] = np.sqrt(sum)
                else:
                    A[i*n+j] = 0
                    raise ValueError(f"Row {i} in matrix not spd!")
            else:
                if A[j*n+j] > 0:
                    A[i*n+j] = sum/A[j*n+j]
                else:
                    raise ValueError("Matrix diagonal is 0")

    for i in range(0, n):
        for j in range(i + 1, n):
            A[i*n+j] = 0

    return A

@njit()
def covariance_matrix(rho0, phi0, k2, chi = 1.1, T = 0.5, kappa = 0.01, temperature = 1e-7, tau_r = 1, tau_p = 1, Gamma = 1):
    ndof = 38
    Q = ndof//2
    kT = temperature
    
    cs2 = sound_speed_square(T) + kappa*k2*rho0
    mu_rho = calculate_dmup_drho(rho0, phi0, chi, T)
    mu_phi = calculate_dmup_dphi(rho0, phi0, chi, T) + k2*kappa
    p_phi = k2*kappa*phi0

    lambdaLB_r = -1. / tau_r
    lambdaLB_p = -1. / tau_p

    lambda_r = -lambdaLB_r * (2 + lambdaLB_r) / 2
    lambda_p = -lambdaLB_p * (2 + lambdaLB_p) / 2
    lambda_rp = -lambdaLB_r * (2 + lambdaLB_p) / 2
    lambda_pr = -lambdaLB_p * (2 + lambdaLB_r) / 2

    Xi = np.zeros((ndof * ndof,), dtype=float)

    Xi[5 * ndof + 5] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[6 * ndof + 6] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[7 * ndof + 7] = 2. * Gamma * kT / rho0 * lambda_p
    Xi[8 * ndof + 8] = 2. * kT * rho0 * (5 - 9 * cs2) * lambda_r
    Xi[9 * ndof + 9] = 8. * kT * rho0 * lambda_r
    Xi[10 * ndof + 10] = (8.0 / 3.0) * kT * rho0 * lambda_r
    Xi[11 * ndof + 11] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[12 * ndof + 12] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[13 * ndof + 13] = (2.0 / 3.0) * kT * rho0 * lambda_r
    Xi[14 * ndof + 14] = 4. * kT * rho0 * lambda_r
    Xi[15 * ndof + 15] = 4. * kT * rho0 * lambda_r
    Xi[16 * ndof + 16] = 4. * kT * rho0 * lambda_r
    Xi[17 * ndof + 17] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[18 * ndof + 18] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 0) * ndof + (Q + 0)] = (4.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 1) * ndof + (Q + 1)] = 18. * kT * rho0 * (1 - cs2) * lambda_r
    Xi[(Q + 2) * ndof + (Q + 2)] = 8. * kT * rho0 * lambda_r
    Xi[(Q + 3) * ndof + (Q + 3)] = (8.0 / 3.0) * kT * rho0 * lambda_r
    Xi[(Q + 4) * ndof + (Q + 4)] = 2. * Gamma * kT / rho0 * (-9 * Gamma * mu_phi + 5) * lambda_p
    Xi[(Q + 5) * ndof + (Q + 5)] = 8. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 6) * ndof + (Q + 6)] = (8.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 7) * ndof + (Q + 7)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 8) * ndof + (Q + 8)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 9) * ndof + (Q + 9)] = (2.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 10) * ndof + (Q + 10)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 11) * ndof + (Q + 11)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 12) * ndof + (Q + 12)] = 4. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 13) * ndof + (Q + 13)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 14) * ndof + (Q + 14)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 15) * ndof + (Q + 15)] = (4.0 / 3.0) * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 16) * ndof + (Q + 16)] = 18. * Gamma * kT / rho0 * (-Gamma * mu_phi + 1) * lambda_p
    Xi[(Q + 17) * ndof + (Q + 17)] = 8. * Gamma * kT / rho0 * lambda_p
    Xi[(Q + 18) * ndof + (Q + 18)] = (8.0 / 3.0) * Gamma * kT / rho0 * lambda_p

    Xi[8 * ndof + (Q + 1)] = 6. * kT * rho0 * (3 * cs2 - 1) * lambda_r
    Xi[(Q + 1) * ndof + 8] = 6. * kT * rho0 * (3 * cs2 - 1) * lambda_r

    Xi[(Q + 4) * ndof + (Q + 16)] = 6. * Gamma * kT / rho0 * (3 * Gamma * mu_phi - 1) * lambda_p
    Xi[(Q + 16) * ndof + (Q + 4)] = 6. * Gamma * kT / rho0 * (3 * Gamma * mu_phi - 1) * lambda_p

    Xi[8 * ndof + (Q + 4)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho)
    Xi[(Q + 4) * ndof + 8] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(8) * ndof + (Q + 16)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 16) * ndof + 8] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)

    Xi[(Q + 1) * ndof + (Q + 4)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 4) * ndof + (Q + 1)] = 3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 1) * ndof + (Q + 16)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)
    Xi[(Q + 16) * ndof + (Q + 1)] = -3. * kT * (Gamma * mu_phi * (rho0 ** 2) * (3 * cs2 - 1) * mu_rho * lambda_pr + cs2 * (3 * Gamma * mu_phi - 1) * p_phi * lambda_rp) / (cs2 * mu_phi * rho0)

    Xi[(0) * ndof + (Q + 4)] = -3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(Q + 4) * ndof + (0)] = -3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(0) * ndof + (Q + 16)] = 3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr
    Xi[(Q + 16) * ndof + (0)] = 3. * Gamma * kT * rho0 * mu_rho / cs2 * lambda_pr

    Xi[(1) * ndof + (Q + 1)] = 3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(Q + 1) * ndof + (1)] = 3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(1) * ndof + (8)] = -3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(8) * ndof + (1)] = -3. * kT * p_phi / (mu_phi * rho0) * lambda_rp
    Xi[(2) * ndof + (5)] = -phi0 * kT * lambda_pr
    Xi[(3) * ndof + (6)] = -phi0 * kT * lambda_pr
    Xi[(4) * ndof + (7)] = -phi0 * kT * lambda_pr
    Xi[(5) * ndof + (2)] = -phi0 * kT * lambda_pr
    Xi[(6) * ndof + (3)] = -phi0 * kT * lambda_pr
    Xi[(7) * ndof + (4)] = -phi0 * kT * lambda_pr

    return Xi

In [ ]:
@njit
def output_spd_homogenous(k2, CHI, KAPPA, T_IN):
    SPD = np.zeros_like((CHI))

    for x in range(SPD.shape[0]):
        for y in range(SPD.shape[1]):
            for z in range(SPD.shape[2]):
                test_spd = True
                for kval in np.nditer(k2.T):
                    try:
                        Xi = covariance_matrix(1, 0, kval, chi = CHI[x,y,z], T = T_IN[x,y,z], kappa = KAPPA[x, y, z])
                        cholesky_decomp(Xi, 38, 5)
                    except:
                        test_spd = False
                        break 
                SPD[x, y, z] = test_spd
                
    return SPD

@njit
def pre_process_inhomogenous_spatially_independent(chi, T, kappa):
    # guess = np.sqrt(3*(chi/2 - T)/T)*1/(np.sqrt(2))
    phi0_guess = np.arange(0.4, 0.8, 0.1)
    for guess in phi0_guess:
        phi0 = newton_c_ver(guess, chi = chi, T = T, tolerance = 1e-4, max_iterations = 1000)
        if phi0 is not None:
            break
    print(phi0)
    # print("phi0:",phi0, " T:",T, " chi:",chi, " kappa:",kappa)

    xi = np.sqrt((0.142*kappa)/((chi/2 - T)-0.31*T*(phi0**2)))
    
    fit_func = lambda x, b:phi0*np.tanh(0.76*(x - b)/xi)
    xraw = np.linspace(-6, 6, 13)
    phi_s = fit_func(xraw, -0.5)
    return phi_s

@njit
def output_spd_inhomogenous_spatially_independent(CHI, T_IN, KAPPA):
    SPD = np.zeros_like((CHI))
    test_spd = True

    for x in range(SPD.shape[0]):
        for y in range(SPD.shape[1]):
            for z in range(SPD.shape[2]):
                chi = CHI[x,y,z]
                T = T_IN[x,y,z]
                kappa = KAPPA[x,y,z]
                phi_s = pre_process_inhomogenous_spatially_independent(chi, T, kappa)
                for phi in np.nditer(phi_s):
                    try:
                        Xi = covariance_matrix(1, phi, 0, chi = chi, T = T, kappa = kappa, tau_r = 0.7886751345948129)
                        cholesky_decomp(Xi, 38, 5)
                    except:
                        test_spd = False
            
                SPD[x,y,z] = test_spd

    return SPD

In [ ]:
chi = 0.6
T = 0.3*chi
kappa = 0.0144

phi_s = pre_process_inhomogenous_spatially_independent(chi, T, kappa)
test_spd = True
for phi in np.nditer(phi_s):
    try:
        Xi = covariance_matrix(1, phi, 0, chi = chi, T = T, kappa = kappa, tau_r = 0.7886751345948129)
        cholesky_decomp(Xi, 38, 5)
    except:
        test_spd = False

test_spd

#### Plotting

In [ ]:
def make_plot_homogenous(points = 10, L = 16):
    chimin = 0.2
    chimax = 1.2

    kappamin = 0.0
    kappamax = 0.05

    propmin = 0.5
    propmax = 0.8

    chi_s = np.linspace(chimin, chimax, points)
    kappa_s = np.linspace(kappamin, kappamax, points)
    prop_s = np.linspace(propmin, propmax, points)
    # prop_s[points//2] = 0.5

    CHI, KAPPA, PROP = np.meshgrid(*[chi_s, kappa_s, prop_s])
    T_IN = CHI*PROP

    freqs = fft.fftshift(fft.fftfreq(L))
    kx, ky, kz = np.meshgrid(*tuple([2*np.pi*freqs for L in [L, L, L]]), indexing='ij')
    k2 = lattice_fourier_laplacian(kx, ky, kz)
    
    SPD = output_spd_homogenous(k2, CHI, KAPPA, T_IN)
    np.savez("spd_covariance_homogenous_matrix.npz", chi = CHI, kappa = KAPPA, T = T_IN, SPD = SPD)

    return 1

In [ ]:
import os

if not os.path.exists("spd_covariance_homogenous_matrix.npz"):
    make_plot_homogenous(points = 50)

# make_plot(points = 10)
# make_plot_homogenous(points = 10)

FILE_IN = np.load("spd_covariance_homogenous_matrix.npz")
CHI = FILE_IN['chi']
KAPPA = FILE_IN['kappa']
T_IN = FILE_IN['T']
SPD = FILE_IN['SPD']
PROPS = T_IN/CHI

fig = plt.figure(figsize = (6, 6)) 
ax = fig.add_subplot(projection='3d')
ax.set_proj_type('ortho')
colors = np.empty(SPD.shape, dtype=object)
colors[SPD == 0] = "w"
colors[SPD == 1] = "lime"

out = ax.voxels(CHI, T_IN, KAPPA, SPD[:-1, :-1, :-1], facecolors=colors[:-1, :-1, :-1], edgecolor='k')
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$T_{in}$")
ax.set_zlabel(r"$\kappa$")
fig.tight_layout()

# ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)
ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)

ax.set_title("Homogenous system")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (4, 4))

slc_plot = np.s_[10, :, :]
im = ax.contourf(CHI[slc_plot], PROPS[slc_plot], SPD[slc_plot], levels = 1, colors = ['tab:red', 'tab:green'])
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$T/\chi$")
curr_prop = np.unique(KAPPA[slc_plot])[0]
ax.set_title(f"$\kappa = {curr_prop:.3f}$")

fig.tight_layout()
fig.savefig(f"./{figures}/valid_homogenous_parameters.png", dpi = 300)

In [ ]:
def make_plot_inhomogenous(points = 10, L = 16):
    chimin = 0.3
    chimax = 1.0

    kappamin = 0.01
    kappamax = 0.05

    propmin = 0.35
    propmax = 0.49

    chi_s = np.linspace(chimin, chimax, points)
    kappa_s = np.linspace(kappamin, kappamax, points)
    prop_s = np.linspace(propmin, propmax, points)

    CHI, KAPPA, PROP = np.meshgrid(*[chi_s, kappa_s, prop_s])
    T_IN = CHI*PROP

    SPD = output_spd_inhomogenous_spatially_independent(CHI, T_IN, KAPPA)
    np.savez("spd_covariance_inhomogenous_matrix.npz", chi = CHI, kappa = KAPPA, T = T_IN, SPD = SPD)

    return 1

In [ ]:
import os

# if not os.path.exists("spd_covariance_inhomogenous_matrix.npz"):
    # make_plot_inhomogenous(points = 10)

make_plot_inhomogenous(points = 50)
FILE_IN = np.load("spd_covariance_inhomogenous_matrix.npz")
CHI = FILE_IN['chi']
KAPPA = FILE_IN['kappa']
T_IN = FILE_IN['T']
SPD = FILE_IN['SPD']
PROPS = T_IN/CHI

fig = plt.figure(figsize = (6, 6)) 
ax = fig.add_subplot(projection='3d')
ax.set_proj_type('ortho')
colors = np.empty(SPD.shape, dtype=object)
colors[SPD == 0] = "w"
colors[SPD == 1] = "lime"

out = ax.voxels(CHI, T_IN, KAPPA, SPD[:-1, :-1, :-1], facecolors=colors[:-1, :-1, :-1], edgecolor='k')
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$T_{in}$")
ax.set_zlabel(r"$\kappa$")

# ax.view_init(30, 60, 0) # ax.view_init(elev, azim, roll)
ax.view_init(30, 60, 0)

ax.set_title("Inhomogenous system")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (3, 3))

slc_plot = np.s_[25, :, :]
im = ax.contourf(CHI[slc_plot], PROPS[slc_plot], SPD[slc_plot], levels = 1, colors = ['tab:green'])
ax.set_xlabel(r"$\chi$")
ax.set_ylabel(r"$T/\chi$")
curr_prop = np.unique(KAPPA[slc_plot])[0]
ax.set_title(f"$\kappa = {curr_prop:.3f}$")

fig.tight_layout()
fig.savefig(f"./{figures}/valid_inhomogenous_parameters.png", dpi = 300)